# Power System Analysis 2025/26 @ IST Work assessment

When running the code, always run all cells!

## Question 1.

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

First, we read the excel file with the network data and create a dictionary of datasets, one for each sheet. As we can see from the output of dfs.keys(), we now have five datasets: 'Nodes', 'Transformers', 'Generators', 'Lines' and 'Loads'.

In [2]:
path = "network_data.xlsx"
xls = pd.ExcelFile(path)

# Dictionary to hold DataFrames
dfs = {}

for sheet in xls.sheet_names:
    dfs[sheet] = pd.read_excel(xls, sheet_name=sheet)

# Display the keys to verify loading
dfs.keys()

dict_keys(['Nodes', 'Transformers', 'Generators', 'Lines', 'Loads'])

The next step is to properly define the base system, as well a the Fortescue matrix to use in later calculations. The base currents will be calculated based on the "Nodes" dataset, which presents the base voltages for each node of the system. Additionally, the base power is set to a fixed 100MVA.

Apart from this, the Resistances and Reactances were combined into a single Impedance collumn in all the datasets for easier handling - and converted from Ohm to per-unit when necessary.

In [3]:
# Define base system:

f = 50 # Hz
Sb=100e6 # W (100 MVA)

# Compute base current
dfs["Nodes"]["Base Current [pu]"] = Sb/(math.sqrt(3)*dfs["Nodes"]["Base Voltage [V]"]) # A

# Transformation matrix
a = np.exp(2j * np.pi / 3)
A = np.array([[1, 1, 1],
              [1, a**2, a],
              [1, a, a**2]])

Z_base = (dfs["Nodes"]["Base Voltage [V]"]**2)/Sb # Ohm

# Maximum fault parameters
tf_max = 2 # seconds
Zf_max_ohm = 40 # Ohm
Zf_max_pu = Zf_max_ohm/Z_base # pu

# Compute the network impedances in pu

# Convert to the correct base 
dfs["Transformers"]["Z [pu]"] = (
    (dfs["Transformers"]["R [pu]"] + 1j * dfs["Transformers"]["X [pu]"])
    * (Sb / (dfs["Transformers"]["Power [MVA]"] * 1e6))
)

dfs["Generators"]["Z [pu]"] = (
    (dfs["Generators"]["R [pu]"] + 1j * dfs["Generators"]["X [pu]"])
    * (Sb / (dfs["Generators"]["Power [MVA]"] * 1e6))
)

# Joins lines resistance and reactance into a complex impedance for easier handling

dfs["Lines"]["Z1 [ohm]"] = dfs["Lines"]["R1 [ohm]"] + dfs["Lines"]["X1 [ohm]"]*1j
dfs["Lines"]["Z0 [ohm]"] = dfs["Lines"]["R0 [ohm]"] + dfs["Lines"]["X0 [ohm]"]*1j

dfs["Lines"]["Z1 [pu]"] = dfs["Lines"]["Z1 [ohm]"]/(dfs["Lines"]["Voltage Level [V]"]**2/Sb)
dfs["Lines"]["Z0 [pu]"] = dfs["Lines"]["Z0 [ohm]"]/(dfs["Lines"]["Voltage Level [V]"]**2/Sb)

dfs["Nodes"]

,Node ID Number,Base Voltage [V],Base Current [pu]
0,1,150000,384.900179
1,2,150000,384.900179
2,3,150000,384.900179
3,4,150000,384.900179
4,5,150000,384.900179
5,6,150000,384.900179
6,7,150000,384.900179
7,8,150000,384.900179
8,9,150000,384.900179
9,10,10000,5773.502692


We can now build our primitive Impedance and Admittance matrices. Since all the transformers in this system have Ynd windings, none of them outright blocks zero sequence current to flow into the Y winding. As such, they should all be included in the primitive zero sequence matrices, since they allow zero sequence current to flow to ground, and as such, represent an impedance in the zero sequence network. The delta side of the transformers on the other hand does not allow current to flow, so the generators are not included in the zero sequence network.

Aditionally, no loads were included since load current is usually negligible when performing short-circuit analysis at the grid level.

In [4]:
# Primitive Impedance matrices
# Created by placing the impedances of each element in the diagonal of a matrix, ordered lines - transformers - generators

Zprim1 = np.diag(
    dfs["Lines"]["Z1 [pu]"].tolist()
    + dfs["Transformers"]["Z [pu]"].tolist()
    + dfs["Generators"]["Z [pu]"].tolist()
)

# The zero-sequence primitive impedance matrix does not include generators, since they are not modeled in zero-sequence

Zprim0 = np.diag(
    dfs["Lines"]["Z0 [pu]"].tolist()
    + dfs["Transformers"]["Z [pu]"].tolist()
)

# Primitive admittance matrices
# Created by inverting the primitive impedance matrices, since [Y] = [Z]^-1

Yprim1 = np.diag(1 / np.diag(Zprim1))
Yprim0 = np.diag(1 / np.diag(Zprim0))

Now we must create the constrain matrices for the zero and positive sequence. These matrices will have a dimmension of 12x20 and 12x17 respectively (number of nodes x number of branches). The main difference between the two sequences will be that, for the zero sequence, the generators will not be included (since they're connected via a delta winding).

Aditionally, I have defined the currents to always flow from the lowest number node to the highest number node.

Since the positive sequence network has the exact same topology as the actual network, we can simply grab the node connections in our datasets to build the constrain matrix:

In [5]:
# Construct the positive sequence constrain matrix C1
C1 = np.zeros((20, 12), dtype=int)  # 20 branches, 12 nodes

# Lines
for branch_idx, row in dfs["Lines"].iterrows():
    node_a = int(row["Node A"]) - 1  # 0-based indexing
    node_b = int(row["Node B"]) - 1

    C1[branch_idx, node_a] = -1
    C1[branch_idx, node_b] =  1

# Transformers
start_row = len(dfs["Lines"])  # where transformer rows begin
for trafo_idx, row in dfs["Transformers"].iterrows():
    node_h = int(row["Node H"]) - 1
    node_x = int(row["Node X"]) - 1
    row_idx = start_row + trafo_idx
    C1[row_idx, node_h] = -1
    C1[row_idx, node_x] =  1


# Generators
start_row += len(dfs["Transformers"])
for gen_idx, row in dfs["Generators"].iterrows():
    node = int(row["Connecting Nodes"]) - 1
    row_idx = start_row + gen_idx
    C1[row_idx, node] = 1


print(C1)

[[-1  1  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  1  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  1  0  0  0  0  0  0]
 [ 0  0 -1  0  1  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  1  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  1  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1  1  0  0  0]
 [-1  0  0  0  0  0  0  0  0  1  0  0]
 [ 0 -1  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0 -1  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1]]


Now for the negative sequence, we have to keep in mind that the transformers are connected to neutral instead of the node on the delta side, which for this case is always the node on collumn "Node X". Aditionally, to keep the convention set that current always flows from low node to high node, and assuming the neutral is node 0, I considered that the "Node H" column now inputted a 1 instead of a -1. We can construct the constrain matrix the same way:

In [6]:
# Construct the positive sequence constrain matrix C0
C0 = np.zeros((17, 12), dtype=int)  # 17 branches, 12 nodes

# Lines
for branch_idx, row in dfs["Lines"].iterrows():
    node_a = int(row["Node A"]) - 1  # 0-based indexing
    node_b = int(row["Node B"]) - 1

    C0[branch_idx, node_a] = -1
    C0[branch_idx, node_b] =  1

# Transformers
start_row = len(dfs["Lines"])  # where transformer rows begin
for trafo_idx, row in dfs["Transformers"].iterrows():
    node_h = int(row["Node H"]) - 1
    row_idx = start_row + trafo_idx
    C0[row_idx, node_h] = 1


print(C0)

[[-1  1  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  1  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  1  0  0  0  0  0  0]
 [ 0  0 -1  0  1  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  1  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  1  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1  1  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0]]


In [7]:
# Check Lines
line_Z = dfs["Lines"]["Z1 [pu]"].tolist()
for i, z in enumerate(line_Z):
    assert Zprim1[i, i] == z, f"Mismatch in line {i}"

# Check Transformers
trafo_Z = dfs["Transformers"]["Z [pu]"].tolist()
for i, z in enumerate(trafo_Z):
    idx = len(line_Z) + i
    assert Zprim1[idx, idx] == z, f"Mismatch in transformer {i}"

# Check Generators
gen_Z = dfs["Generators"]["Z [pu]"].tolist()
for i, z in enumerate(gen_Z):
    idx = len(line_Z) + len(trafo_Z) + i
    assert Zprim1[idx, idx] == z, f"Mismatch in generator {i}"

print("All Zprim1 entries match C1 row order")

All Zprim1 entries match C1 row order


We now have everything needed to compute the node impedance and admittance matrices, using the following equations:
$$
[\overline{Y}] = [K]^{t}[\overline{y}][K]
$$

$$
[\overline{Z}]^{-1} = [\overline{Y}]
$$

In [8]:
# Positive Sequence Node Admittance Matrix Computation
Y1 = C1.T @ Yprim1 @ C1

# Zero Sequence Node Admittance Matrix Computation
Y0 = C0.T @ Yprim0 @ C0

# Node Impedance Matrices
Z1 = np.linalg.inv(Y1)
Z0 = np.linalg.pinv(Y0)

### Three Phase Fault

For a three phase fault, we have equation to calculate the fault current at each node:
$$
\overline{I}_{1-\Delta k} = \frac{\overline{V}_{1 - pF}}{\overline{Z}_{1-kk}}
$$

Which, since $\overline{V}_{1 - pF}$ is always 1 in per-unit, becomes:

$$
\overline{I}_{1-\Delta k} = \overline{Z}_{1-kk}^{-1}
$$

Additionally, it's important to state that for a three phase fault, the current in all 3 phases (a, b, c) is the same and equal to the positive sequence current.

In [9]:
Zkk_TP = np.diag(Z1)              # Array of self-impedances
If_pu_TP = 1 / Zkk_TP              # Array of fault currents in pu

Tp_TP = np.where(Zkk_TP.real != 0, Zkk_TP.imag / Zkk_TP.real, np.nan)

I_base = dfs["Nodes"]["Base Current [pu]"].values   # Array of base currents

I_seq = np.vstack([np.zeros_like(If_pu_TP), If_pu_TP, np.zeros_like(If_pu_TP)])     # Build matrix with I0 I1 and I2 stacked vertically

# Phase currents in pu
I_abc_pu = (A @ I_seq)

# Convert to Amperes
I_abc_A = I_abc_pu * I_base

# Compute RMS and phase angle
I_abc_rms = np.abs(I_abc_A)

I_abc_angle = np.angle(I_abc_A, deg=True)

# Build DataFrame
n_nodes = I_abc_A.shape[1]
df_TP = pd.DataFrame({
    "Node": np.repeat(np.arange(n_nodes), 3),
    "TP Phase": np.tile(["A", "B", "C"], n_nodes),
    "TP Irms[A]": I_abc_rms.T.flatten(),
    "TP Phase[deg]": I_abc_angle.T.flatten(),
    "TP Tp[s]": np.repeat(Tp_TP, 3)
})

df_TP = df_TP.round({"TP Irms[A]": 1, "TP Phase[deg]": 1, "TP Tp[s]": 3})


### Single Line to Ground Fault

Similarly, we compute the SLG fault currents using the following equation:

$$\bar{I}_{0-\Delta k} = \bar{I}_{1-\Delta k} = \bar{I}_{2-\Delta k} = \frac{\bar{V}_{1-pF}}{\bar{Z}_{0-kk} + \bar{Z}_{1-kk} + \bar{Z}_{2-kk} + 3R_f} = \frac{\bar{V}_{1-pF}}{\bar{Z}_T}$$

In [10]:
ZT_SLG = np.diag(Z1) + np.diag(Z1) + np.diag(Z0) + 3*Zf_max_pu
ZT_SLG = np.asarray(ZT_SLG)  # make sure it's a NumPy array

If_pu_SLG = 1.0 / ZT_SLG
Tp_SLG = np.where(ZT_SLG.real != 0, ZT_SLG.imag / ZT_SLG.real, np.nan)

# Fault current with zero fault impedance (bolted fault)
ZT_SLG_zero = np.diag(Z1) + np.diag(Z1) + np.diag(Z0)
ZT_SLG_zero = np.asarray(ZT_SLG_zero)

If_pu_SLG_zero = 1.0 / ZT_SLG_zero
Tp_SLG_zero = np.where(ZT_SLG_zero.real != 0, ZT_SLG_zero.imag / ZT_SLG_zero.real, np.nan)


# Base currents
I_base = dfs["Nodes"]["Base Current [pu]"].values  # A

# Stack I0, I1, I2 for each node
I_seq = np.vstack([If_pu_SLG, If_pu_SLG, If_pu_SLG])         # SLG with Zf
I_seq_zero = np.vstack([If_pu_SLG_zero, If_pu_SLG_zero, If_pu_SLG_zero])  # bolted fault

# Phase currents in pu
I_abc_pu = (A @ I_seq)
I_abc_pu_zero = (A @ I_seq_zero)

# Convert to Amperes
I_abc_A = I_abc_pu * I_base
I_abc_A_zero = I_abc_pu_zero * I_base

# Compute RMS and phase angle
I_abc_rms = np.abs(I_abc_A)
I_abc_rms_zero = np.abs(I_abc_A_zero) 

I_abc_angle = np.angle(I_abc_A, deg=True)
I_abc_angle_zero = np.angle(I_abc_A_zero, deg=True)

# Build DataFrame
n_nodes = I_abc_A.shape[1]
df_SLG = pd.DataFrame({
    "Node": np.repeat(np.arange(n_nodes), 3),
    "SLG Phase": np.tile(["A", "B", "C"], n_nodes),
    "SLG Irms Min[A]": I_abc_rms.T.flatten(),
    "SLG Phase Min[deg]": I_abc_angle.T.flatten(),
    "SLG Tp Min[s]": np.repeat(Tp_SLG, 3),
    "SLG Irms Max[A]": I_abc_rms_zero.T.flatten(),
    "SLG Phase Max[deg]": I_abc_angle_zero.T.flatten(),
    "SLG Tp Max[s]": np.repeat(Tp_SLG_zero, 3)
})

df_SLG = df_SLG.round({"SLG Irms Min[A]": 1, "SLG Phase Min[deg]": 1, "SLG Tp Min[s]": 3,
                       "SLG Irms Max[A]": 1, "SLG Phase Max[deg]": 1, "SLG Tp Max[s]": 3})

### Line to Line fault

And finally, for the line to line fault:

$$\bar{I}_{1-\Delta k} = - \bar{I}_{2-\Delta k} = \frac{\bar{V}_{1-pF}}{\bar{Z}_{1-kk} + \bar{Z}_{2-kk} + 3R_f} = \frac{\bar{V}_{1-pF}}{\bar{Z}_T}$$

In [11]:
# Fault current with Zf_max_pu
ZT_LL = np.diag(Z1) + np.diag(Z1) + Zf_max_pu
ZT_LL = np.asarray(ZT_LL)  # make sure it's a NumPy array
If_pu_LL = 1.0 / ZT_LL  # pu assuming V_prefault = 1 pu
Tp_LL = np.where(ZT_LL.real != 0, ZT_LL.imag / ZT_LL.real, np.nan)

# Base currents
I_base = dfs["Nodes"]["Base Current [pu]"].values  # A

# Stack I0, I1, I2 for each node
I_seq = np.vstack([np.zeros_like(If_pu_LL), If_pu_LL, -If_pu_LL])

# Phase currents in pu
I_abc_pu = (A @ I_seq)

# Convert to Amperes
I_abc_A = I_abc_pu * I_base


# Compute RMS and phase angle
I_abc_rms = np.abs(I_abc_A)

I_abc_angle = np.angle(I_abc_A, deg=True)

# Build DataFrame
n_nodes = I_abc_A.shape[1]
df_LL = pd.DataFrame({
    "Node": np.repeat(np.arange(n_nodes), 3),
    "LL Phase": np.tile(["A", "B", "C"], n_nodes),
    "LL Irms[A]": I_abc_rms.T.flatten(),
    "LL Phase[deg]": I_abc_angle.T.flatten(),
    "LL Tp[s]": np.repeat(Tp_LL, 3)
})

df_LL = df_LL.round({"LL Irms[A]": 1, "LL Phase[deg]": 1, "LL Tp[s]": 3})


### Final Table 1:

In [12]:
df_TP = df_TP.iloc[::3]
df_SLG = df_SLG.iloc[::3]
df_LL = df_LL.iloc[1::3]

df_TP["Node"] = df_TP["Node"] + 1
df_SLG["Node"] = df_SLG["Node"] + 1
df_LL["Node"] = df_LL["Node"] + 1

combined_df = df_TP.merge(df_SLG, on="Node").merge(df_LL, on="Node")

combined_df.drop(combined_df.index[-3:], inplace=True)

combined_df

,Node,TP Phase,TP Irms[A],TP Phase[deg],TP Tp[s],SLG Phase,SLG Irms Min[A],SLG Phase Min[deg],SLG Tp Min[s],SLG Irms Max[A],SLG Phase Max[deg],SLG Tp Max[s],LL Phase,LL Irms[A],LL Phase[deg],LL Tp[s]
0,1,A,16672.0,-88.6,39.708,A,2147.0,-6.1,0.106,20295.6,-88.5,38.700,B,3607.5,-104.5,0.258
1,2,A,10611.6,-86.9,18.363,A,2113.5,-9.8,0.172,12450.3,-86.8,18.160,B,3407.8,-111.7,0.399
2,3,A,5389.7,-84.5,10.338,A,1800.8,-27.9,0.529,3826.6,-83.6,8.970,B,2795.0,-126.6,0.742
3,4,A,4789.1,-83.9,9.281,A,1719.7,-31.0,0.601,3314.7,-83.0,8.187,B,2644.2,-129.3,0.820
4,5,A,4286.8,-84.1,9.696,A,1640.1,-34.8,0.694,2858.0,-83.6,8.920,B,2512.6,-132.3,0.910
5,6,A,3973.5,-84.8,11.019,A,1609.7,-36.7,0.745,2682.2,-84.4,10.201,B,2428.4,-134.7,0.988
6,7,A,8391.7,-87.5,23.013,A,2096.4,-12.0,0.213,10068.0,-87.3,21.150,B,3275.1,-116.8,0.504
7,8,A,1322.1,-83.6,8.890,A,738.1,-63.6,2.017,818.5,-83.4,8.668,B,1062.4,-157.2,2.382
8,9,A,983.7,-84.2,9.929,A,567.1,-69.0,2.611,604.1,-84.1,9.751,B,813.3,-161.8,3.039


## Question 2.

Insulation: $175kV_{RMS}$
This value is picked to give some wiggle room on the 150kV rating of the lines, and was selected from the IEC 60071 rated insulation values.


Turn ratio: $1500:1$
To convert 150kV to 100V (line-to-line).

Accuracy class: $3P$

## Question 3.

Here lets assume one current transformer per impedance, so a total of 17 current transformers (not considering generators).
We can see that the fault currents are much higher in the nodes connected to the generator transformers so let's define two classes of current transformers:
- Class A for before and after nodes 1, 2 and 3
- Class B everywhere else

In [13]:
# Find the maximum Isc among all fault types and nodes
group_A = combined_df.iloc[:3]
group_B = combined_df.iloc[3:]

IscA_max = max(group_A['TP Irms[A]'].max(), group_A['SLG Irms Max[A]'].max(), group_A['LL Irms[A]'].max())

IscB_max = max(group_B['TP Irms[A]'].max(), group_B['SLG Irms Max[A]'].max(), group_B['LL Irms[A]'].max())

print("IscA_max: " + str(IscA_max))
print("IscB_max: " + str(IscB_max))

# Find the maximum Tp among all fault types and nodes

TpA_max = max(group_A['TP Tp[s]'].max(), group_A['SLG Tp Max[s]'].max(), group_A['LL Tp[s]'].max())
TpB_max = max(group_B['TP Tp[s]'].max(), group_B['SLG Tp Max[s]'].max(), group_B['LL Tp[s]'].max())

print("TpA_max: " + str(TpA_max))
print("TpB_max: " + str(TpB_max))

Ts = 3 # s
t_sat = 0.04 # s
R_B = 5  # Ohm
R_CT = 6  # Ohm
I_2n = 1  # A

IscA_max: 20295.6
IscB_max: 10068.0
TpA_max: 39.708
TpB_max: 23.013


Now, to compute the turn ratio of the transformer we need to consider the maximum current that would flow through it under normal conditions. For this I considered:
- For group A, the maximum generator S
- For group B, the maximum load S

In [14]:
NA = dfs["Transformers"]["Power [MVA]"].max() * 1e6 / 150000 / I_2n
NB = dfs["Loads"]["Load MW"].max() * 1e6 / 150000 / I_2n

# Define a function to round up to the nearest allowed base value
def round_up_to_allowed(value, bases):
    for exponent in range(0, 20):
        for b in sorted(bases):
            candidate = b * (10 ** exponent)
            if candidate >= value:
                return candidate
    raise ValueError("Value too large for allowed bases")

# Allowed bases
bases = [10, 12.5, 15, 20, 25, 30, 40, 50, 60, 75]

# Use 1.2, the safety factor for NA
NA = round_up_to_allowed(NA*1.2, bases)

# Use 2 for NB since loads vary way more
NB = round_up_to_allowed(NB*2, bases)

print("NA: " + str(NA))
print("NB: " + str(NB))

NA: 3000
NB: 600


We define a function to size the current transformer (CT).  

1. **Rated Symmetric Short-Circuit Factor**  
   $$
   K_{SSC} = \frac{I_{SC,\text{max}}}{N}
   $$
   where $I_{SC,\text{max}}$ is the maximum short-circuit current and $N$ is the CT turns ratio.  

2. **Transient Factor**  
   $$
   K_{tf} = \frac{\omega T_P T_S}{T_P - T_S} \left(e^{-\frac{t}{T_P}} - e^{-\frac{t}{T_S}}\right)
   $$  
   where $T_P$ and $T_S$ are the primary and secondary time constants, and $\omega = 2\pi f$.  

3. **Saturation Electromotive Force**  
   $$
   E_{mf,\;sat} = K_{SSC} \times K_{tf} \times I_{2n} \times (R_B + R_{CT})
   $$  
   with $R_B = 5\ \Omega$, $R_{CT} = 6\ \Omega$, and nominal secondary current $I_{2n} = 1\ \mathrm{A}$.  

4. **Core Cross-Section**  
   $$
   A = \frac{E_{mf,\;sat}}{\frac{2\pi}{\sqrt{2}} \times B_{sat} \times f \times N}
   $$  
   where $B_{sat}$ is the saturation flux density of the core material, $f$ is the system frequency, and $N$ is the number of turns.  


In [15]:
def size_itrans(Isc_max, N, Tp_max, R_B, R_CT, I_2n, Ts, t_sat):
    k_ssc = math.ceil(Isc_max / N)

    k_tf = (2 * np.pi * f * Tp_max * Ts) / (Tp_max - Ts) * (np.exp(-t_sat / Tp_max) - np.exp(-t_sat / Ts))
    
    k_tf = math.ceil(k_tf)

    E_mf_sat = k_ssc * k_tf * I_2n * (R_B + R_CT)

    A = E_mf_sat / ((2 * np.pi / np.sqrt(2)) * 1.4 * f * N)
    A = A * 10000  # cm^2

    print("K_ssc: " + str(k_ssc))
    print("K_tf: " + str(k_tf))
    print("A: " + str(A))
    print("")
    print(str(N) + ":1 Rb = " + str(R_B) + "; class = TPX " + str(k_ssc) + "x" + str(k_tf) + " RCT < " + str(R_CT))
    print("")

size_itrans(IscA_max, NA, TpA_max, R_B, R_CT, I_2n, Ts, t_sat)
size_itrans(IscB_max, NB, TpB_max, R_B, R_CT, I_2n, Ts, t_sat)

K_ssc: 7
K_tf: 13
A: 10.728769434205514

3000:1 Rb = 5; class = TPX 7x13 RCT < 6

K_ssc: 17
K_tf: 13
A: 130.27791455820983

600:1 Rb = 5; class = TPX 17x13 RCT < 6



As we can see above, for class A we got:
$$
3000:1; Rb = 5; class = TPX; 7\times13; RCT < 6 \Omega
$$

and for class B:
$$
600:1; Rb = 5; class = TPX; 17x13; RCT < 6 \Omega
$$